In [23]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

In [24]:
advertiser_id = '3427'
path = "/Users/denniscimorosi/Desktop/IPinYou/make-ipinyou-data-master/" + advertiser_id + "/"
test_name = "test.log.txt"
train_name = "train.log.txt"

In [25]:
train_df = pd.read_csv(path + train_name, delimiter='\t')
test_df = pd.read_csv(path + test_name, delimiter='\t')

In [19]:
upsample = False
encoding = 'one-hot'

In [20]:
train_df = train_df.drop(['bidid', 'logtype', 'ipinyouid', 'IP', 'adexchange',
                          'urlid', 'url', 'slotid', 'slotwidth', 'slotheight', 'slotvisibility',
                          'slotformat', 'creative', 'keypage', 'advertiser','usertag'], axis=1)

test_df = test_df.drop(['bidid', 'logtype', 'ipinyouid', 'IP', 'adexchange',
                          'urlid', 'url', 'slotid', 'slotwidth', 'slotheight', 'slotvisibility',
                          'slotformat', 'creative', 'keypage', 'advertiser', 'usertag', 'nclick', 'nconversation'], axis=1)

In [26]:
test_df['click'].sum()

366

In [21]:
train_df.shape[0]

2593765

In [7]:
# split useragent columns into os and browser 
def split_os_browser(df):
    def extract_os(x):
        return x.split("_")[0]

    def extract_browser(x):
        return x.split("_")[1]

    vfun = np.vectorize(extract_os)
    df['os'] = vfun(df['useragent'])

    vfun = np.vectorize(extract_browser)
    df['browser'] = vfun(df['useragent'])

    df.drop(['useragent'], inplace=True, axis=1)

    return df

train_df = split_os_browser(train_df)
test_df = split_os_browser(test_df)

In [8]:
# nan domains handling
train_df['domain'] = train_df['domain'].fillna('unknown')
test_df['domain'] = test_df['domain'].fillna('unknown')

def aggregate(x):
    return 'other' if x not in to_keep else x

# aggregate domains which occur less than 1000 times
domains = train_df.groupby('domain', as_index=False).count()
to_keep = domains[domains['click'] > 1000]['domain']
to_keep = np.array(to_keep)

vfun = np.vectorize(aggregate)
train_df['domain'] = vfun(train_df['domain'])
test_df['domain'] = vfun(test_df['domain'])

# aggregate cities which occur less than 8000 times
cities = train_df.groupby('city', as_index=False).count()
to_keep = domains[domains['click'] > 8000]['city']
to_keep = np.array(to_keep)

vfun = np.vectorize(aggregate)
train_df['city'] = vfun(train_df['city'])
test_df['city'] = vfun(test_df['city']) 

In [9]:
# label encoding
def label_enc(test_df, train_df, feature_name):
    # extract all feature values from both train and test df
    test_feats = test_df[feature_name].unique()
    train_feats = train_df[feature_name].unique()
    all_feats = np.unique(np.concatenate([train_feats, test_feats]))
    # fit label encoder
    encoder = preprocessing.LabelEncoder().fit(all_feats)
    # transform feature
    test_df[feature_name] = encoder.transform(test_df[feature_name])
    train_df[feature_name] = encoder.transform(train_df[feature_name])
    return test_df, train_df

if encoding == 'label':
    test_df, train_df = label_enc(test_df, train_df, 'os')
    test_df, train_df = label_enc(test_df, train_df, 'browser')
    test_df, train_df = label_enc(test_df, train_df, 'domain')

In [10]:
def one_hot_enc(test_df, train_df, feature_name):
    test_cols = []
    train_cols = []

    # extract all feature values from both train and test df
    test_feats = test_df[feature_name].unique()
    train_feats = train_df[feature_name].unique()
    all_feats = np.unique(np.concatenate([train_feats, test_feats]))

    # fit one-hot encoder
    encoder = preprocessing.LabelBinarizer().fit(all_feats)

    transformed = encoder.transform(test_df[feature_name])
    test_ohe = pd.DataFrame(transformed)

    transformed = encoder.transform(train_df[feature_name])
    train_ohe = pd.DataFrame(transformed)

    for col in test_ohe.columns:
        test_cols.append(feature_name + '_' + str(col))
    test_ohe.columns = test_cols
    test_df = test_df.join(test_ohe).drop([feature_name], axis=1)

    for col in train_ohe.columns:
        train_cols.append(feature_name + '_' + str(col))
    train_ohe.columns = train_cols
    train_df = train_df.join(train_ohe).drop([feature_name], axis=1)

    return test_df, train_df

if encoding == 'one-hot':
    test_df, train_df = one_hot_enc(test_df, train_df, 'os')
    test_df, train_df = one_hot_enc(test_df, train_df, 'browser')
    test_df, train_df = one_hot_enc(test_df, train_df, 'domain')
    test_df, train_df = one_hot_enc(test_df, train_df, 'city')
    test_df, train_df = one_hot_enc(test_df, train_df, 'region')


In [11]:
train_df.to_csv(advertiser_id + '.train.final.csv', index = False)
test_df.to_csv(advertiser_id + '.test.final.csv', index = False)

In [12]:
train_df

,click,weekday,hour,timestamp,slotprice,bidprice,payprice,os_0,os_1,os_2,...,region_25,region_26,region_27,region_28,region_29,region_30,region_31,region_32,region_33,region_34
0,0,4,0,20130606000104519,0,238,64,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,4,0,20130606000104552,0,238,127,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,4,0,20130606000104612,0,238,225,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,4,0,20130606000104710,5,249,265,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,4,0,20130606000104786,0,238,41,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970355,0,3,10,20130612103253642,0,238,136,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1970356,0,3,10,20130612103255375,0,238,85,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1970357,0,3,10,20130612103302190,0,238,156,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1970358,0,3,10,20130612103318102,0,238,41,0,0,0,...,0,0,0,0,0,0,0,0,0,0
